In [2]:
import pandas as pd  
import numpy as np  

In [3]:
player_data = pd.read_parquet('../../Data/Player_Matchup_Data.parquet')

In [4]:
player_data_2024 = player_data[player_data['season'] == 2024]

In [9]:
player_data_2024[player_data_2024['owner'] == 'Jose']

,season,week,roster_id,owner,player_id,player_name,position,slot,fantasy_points
2,2024,1,1,Jose,11566,Jayden Daniels,QB,bench,28.16
16,2024,1,1,Jose,2028,Derek Carr,QB,bench,27.30
15,2024,1,1,Jose,11624,Xavier Worthy,WR,bench,20.80
12,2024,1,1,Jose,11631,Brian Thomas,WR,bench,14.70
3,2024,1,1,Jose,9753,Zach Charbonnet,RB,bench,11.10
...,...,...,...,...,...,...,...,...,...
5380,2024,18,1,Jose,11566,Jayden Daniels,QB,starter,4.22
5409,2024,18,1,Jose,6804,Jordan Love,QB,starter,2.76
5385,2024,18,1,Jose,8121,Romeo Doubs,WR,starter,0.00
5393,2024,18,1,Jose,11624,Xavier Worthy,WR,starter,0.00


### Best Possible Lineup
NOTE: This produces the total number of points produced by the most optimal line up, which is great, but maybe it would be a good idea to have the actual optimal lineup outputted as well?

In [10]:
def best_possible_lineup(df):
    total = 0

    def take(pos, n):
        nonlocal df, total
        picked = df[df["position"] == pos].nlargest(n, "fantasy_points")
        total += picked["fantasy_points"].sum()
        df = df.drop(picked.index)

    # Required slots
    take("QB", 1)
    take("RB", 2)
    take("WR", 3)
    take("TE", 1)

    # Flex (RB/WR/TE)
    flex_pool = df[df["position"].isin(["RB", "WR", "TE"])]
    flex = flex_pool.nlargest(2, "fantasy_points")
    total += flex["fantasy_points"].sum()
    df = df.drop(flex.index)

    # Super Flex (QB/RB/WR/TE)
    sf = df[df["position"].isin(["QB", "RB", "WR", "TE"])].nlargest(1, "fantasy_points")
    total += sf["fantasy_points"].sum()

    return total


In [11]:
optimal = (
    player_data_2024
    .groupby(["owner", "week"])
    .apply(best_possible_lineup)
    .reset_index(name="optimal_points")
)

/var/folders/yw/4djwspfd71dd2kz47p89ghvm0000gn/T/ipykernel_9780/3391063217.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  player_data_2024


In [12]:
optimal

,owner,week,optimal_points
0,Brigido,1,206.18
1,Brigido,2,179.88
2,Brigido,3,153.24
3,Brigido,4,191.32
4,Brigido,5,224.72
...,...,...,...
175,Marvin,14,167.92
176,Marvin,15,159.40
177,Marvin,16,180.46
178,Marvin,17,200.80


### Coaching Efficiency (%)
How good was this manager at setting lineups?

In [13]:
actual = (
    player_data_2024[player_data_2024["slot"] == "starter"]
    .groupby(["owner", "week"])["fantasy_points"]
    .sum()
    .reset_index(name="actual_points")
)

efficiency = actual.merge(optimal, on=["owner", "week"])
efficiency["efficiency"] = efficiency["actual_points"] / efficiency["optimal_points"]

In [14]:
efficiency

,owner,week,actual_points,optimal_points,efficiency
0,Brigido,1,158.90,206.18,0.770686
1,Brigido,2,172.58,179.88,0.959417
2,Brigido,3,107.10,153.24,0.698904
3,Brigido,4,139.54,191.32,0.729354
4,Brigido,5,218.62,224.72,0.972855
...,...,...,...,...,...
175,Marvin,14,138.02,167.92,0.821939
176,Marvin,15,138.10,159.40,0.866374
177,Marvin,16,156.56,180.46,0.867561
178,Marvin,17,177.50,200.80,0.883964


### Lineup Risk Profile
Is this manager boom/bust or consistent?

NOTE: This is currently looking at indicudaul players by owner, maybe we should be looking at the aggregated starter lineup points from week to week instead? This way we measure "how consistent owners are from week to week"

In [15]:
volatility = (
    player_data_2024[player_data_2024["slot"] == "starter"]
    .groupby("owner")["fantasy_points"]
    .agg(["mean", "std", "max"])
)

In [16]:
volatility

,mean,std,max
owner,,,
Brigido,16.620222,10.127350,46.10
Bryan,14.080667,8.498162,43.00
Daryl,17.008222,9.994869,51.06
Devonte,16.766444,9.421940,55.40
Gio,12.474556,7.958486,40.50
Jacky,14.070000,8.273201,37.14
Jose,12.908778,8.878193,44.42
Lalo,13.288827,9.139061,37.80
Luis,14.498889,9.615964,57.88


### Positional Decision Quality
"Where does this manager mess up the most?"

In [17]:
qb_mistakes = (
    player_data_2024
    .query("position == 'QB'")
    .groupby(["owner", "week", "slot"])["fantasy_points"]
    .max()
    .unstack()
    .reset_index()
)

qb_mistakes["qb_mistake"] = qb_mistakes["bench"] > qb_mistakes["starter"]


In [20]:
qb_mistakes[qb_mistakes['owner'] == 'Jose']

slot,owner,week,bench,starter,qb_mistake
108,Jose,1,28.16,20.40,True
109,Jose,2,25.82,16.94,True
110,Jose,3,32.06,20.38,True
111,Jose,4,36.16,26.02,True
112,Jose,5,16.60,21.72,False
113,Jose,6,0.00,33.62,False
114,Jose,7,12.06,22.80,False
115,Jose,8,22.18,24.24,False
116,Jose,9,30.54,23.86,True
117,Jose,10,24.46,18.54,True


### Start/Sit Regret Index

In [24]:
efficiency["points_left"] = (
    efficiency["optimal_points"] - efficiency["actual_points"]
)

efficiency["regret_score"] = (
    (efficiency["points_left"] > 30).astype(int) * 2 +
    ((efficiency["points_left"] > 15) & (efficiency["points_left"] <= 30)).astype(int)
)


In [26]:
efficiency[efficiency['owner'] == "Jose"]

,owner,week,actual_points,optimal_points,efficiency,points_left,regret_score
108,Jose,1,119.54,171.16,0.698411,51.62,2
109,Jose,2,101.18,154.16,0.656331,52.98,2
110,Jose,3,183.46,219.14,0.837182,35.68,2
111,Jose,4,145.22,188.48,0.770480,43.26,2
112,Jose,5,156.38,178.58,0.875686,22.20,1
113,Jose,6,140.18,174.48,0.803416,34.30,2
114,Jose,7,118.54,179.66,0.659802,61.12,2
115,Jose,8,118.88,171.12,0.694717,52.24,2
116,Jose,9,110.08,151.10,0.728524,41.02,2
117,Jose,10,96.62,142.80,0.676611,46.18,2


### Consistency Vs. Ceailing Managers
"Who wins with floor vs. ceiling?"

In [22]:
weekly = (
    player_data_2024[player_data_2024["slot"] == "starter"]
    .groupby(["owner", "week"])["fantasy_points"]
    .sum()
    .reset_index()
)

summary = weekly.groupby("owner")["fantasy_points"].agg(
    avg="mean",
    floor="min",
    ceiling="max"
)


In [23]:
summary

,avg,floor,ceiling
owner,,,
Brigido,166.202222,107.10,221.80
Bryan,140.806667,53.06,189.86
Daryl,170.082222,104.72,223.80
Devonte,167.664444,117.92,210.36
Gio,124.745556,43.64,169.38
Jacky,140.700000,89.98,189.10
Jose,129.087778,56.68,183.46
Lalo,132.150000,88.78,191.26
Luis,144.988889,93.80,192.90


9️⃣ Weekly Awards (Auto-Generated)  
You can now automate:  
🏆 Best Lineup (highest actual)  
🧠 Best Coaching (highest efficiency)  
🤡 Worst Coaching  
💀 Worst Bench  
🚀 Highest Ceiling Week   
All trivial idxmax() calls.  

Formalize optimal lineup logic (guaranteed-valid solver)  
Build an award generator  
Add injury-aware adjustments  
Create a manager scorecard  